# PART 3/3: SIMULATING SPECIFIC MOLECULES
# MMM 2025 - 12.3.2025
## Daniele Passerone


The molecules you will explore today are:

1. C2H2 (acetylene)
2. H2 (hydrogen molecule)
3. CH4 (methane)
4. O2 (triplet oxygen)
5. hexatriene (C6H8)
6. Benzene (C6H6)

## Prerequisites

We assume that you already have learned how to run a geo opt workchain as well as an STM/Orbitals run. 
You will have to run them for the molecules listed above, for each of them changing slightly the parameters (for example, number of filled orbitals).
For the analysis of the simulations, you will need to keep track of the pk of the simulation, pk of the trajectory and so on, as explained in the notebook 2/3.


In [2]:
#
# some important Imports...
#
import numpy as np
from ase import Atoms
from ase.io import read
from ase.visualize import view
import matplotlib.pyplot as plt
import nglview as nv
from show_orbitals import all_orbitals

#
# and definitions of visualization functions (see last exercises)
#
def view_structure(structure,myvec=[]):
    t = nv.ASEStructure(structure)
    w = nv.NGLWidget(t, gui=True)
    w.add_unitcell()
    w.add_ball_and_stick()
    w.add_representation('label',label_type='atomindex',color='black')
    w.add_representation('spacefill',selection=myvec,color="blue",radius=0.5)
    return w

def view_trajectory(trajectory,myvec=[]):
    t2 = nv.ASETrajectory(trajectory)
    w2 = nv.NGLWidget(t2, gui=True)
    #w2.add_unitcell()
    w2.add_ball_and_stick()
    w2.add_representation('spacefill',selection=myvec,color="blue",radius=0.5)
    return w2

## 0. Example with the acetylene molecule

The following assumes that you have run the **geo opt** of C2H2 as well as the **SCANNING PROBE MICROSCOPY/ORBITALS** run. The other molecule can be faced in a similar way.

## 1. Visualizing the optimization trajectory 
As shown in the **Notebook 2** you need the pk of the optimization trajectory, that you will insert in the next cell as value of **trajpk**.

In [18]:

%load_ext aiida
%aiida

from aiida.orm import load_node

my_geo_opt_pk = 38 # THE PK OF THE GEO OPT
my_pk = 58 # THE ONE OF THE ORBITALS
my_numberocc = 5 # THE NUMBER OF OCCUPIED ORBITALS IN THE SYSTEM (VALENCE ELECTRONS/2)

workchain = load_node(my_geo_opt_pk)

# Access the TrajectoryData node
trajectory_data = workchain.outputs.output_trajectory  
trajectory_pk = trajectory_data.pk

print(f"The PK of the TrajectoryData is: {trajectory_pk}")

trajectory_c2h2 = trajectory_pk
trajpk = trajectory_c2h2
traj = load_node(trajpk)
symbols = traj.symbols
trajase=[traj.get_step_structure(step-1).get_ase() for step in traj.get_stepids()]
for a in trajase:
    a.set_pbc([False,False,False])

The aiida extension is already loaded. To reload it, use:
  %reload_ext aiida
The PK of the TrajectoryData is: 54


In [4]:
view_trajectory(trajase)

#
# The optimization trajectory is shown.
#

NGLWidget(max_frame=15)

## 2. Visualizing the orbitals
Now we compute the orbitals. To this end we have already finished (see **Notebook 2**) the simulation of the orbitals, and hit the "Cube creation kit" button. Keep the **pk** of the SPM calculation ready.

In the Exercise_5 directory, there is a script "run_cube_from_wfn_acetylene.sh" that is able to take some files from the cp2k SPM simulation (wavefunction) and transform it into the orbital cube files. The important things in this file is the number of occupied and unoccupied orbitals, that should correspond to the ones that you have indicated when launching the SPM AiiDAlab workchain. In the case of acetylene, 10 electrons, 5 occupied orbitals.

In [5]:
!cat run_cube_from_wfn_c2h2.sh

#!/bin/bash -l

DIR="./"

mkdir cubes

/home/jovyan/.local/bin/cp2k-cube-from-wfn  --cp2k_input_file $DIR/aiida.inp \
  --basis_set_file BASIS_MOLOPT \
  --xyz_file $DIR/aiida.coords.xyz \
  --wfn_file $DIR/aiida-RESTART.wfn \
  --output_dir ./cubes/ \
  --n_homo 5 \
  --n_lumo 5 \
  --dx 0.2 \
  --eval_cutoff 14.0 \



## Generating the cubefiles, using the "cube-kit" set of files and the above script. 
Now we are ready to generate the orbital cube files. We replace my_pk below **with the pk of the SPM workchain**, and also the molecule name has to correspond.

In [7]:
#
# Creating the cube file of the orbitals
# 

molecule = 'c2h2'
!rm -Rf ./cube-kit-pk{my_pk}*
!cp /home/jovyan/apps/surfaces/tmp/cube-kit-pk{my_pk}.zip .
!unzip cube-kit-pk{my_pk}.zip
!cp run_cube_from_wfn_{molecule}.sh ./cube-kit-pk{my_pk}
!cd ./cube-kit-pk{my_pk} ; bash run_cube_from_wfn_{molecule}.sh 
!rm -Rf {molecule}_cubes
!mv ./cube-kit-pk{my_pk}/cubes {molecule}_cubes

print ("*****************************\n\nTHE GENERATED FILES ARE:\n")
!ls ./{molecule}_cubes
print ("\n*****************************\n")

Archive:  cube-kit-pk58.zip
  inflating: cube-kit-pk58/BASIS_MOLOPT  
  inflating: cube-kit-pk58/aiida.inp  
  inflating: cube-kit-pk58/aiida.out  
  inflating: cube-kit-pk58/aiida.coords.xyz  
  inflating: cube-kit-pk58/aiida-RESTART.wfn  
R0/1, loading indexes (s0/1) 0:9 / 0:9
eval_cell_n:  [67 50 51]
loc_cell_n:  [71 70 70]
---- Setup: 0.0014
---- Radial calc time : 0.234975
---- Spherical calc time : 0.013309
---- Loc -> loc_morb time : 0.071365
---- loc_morb -> glob time : 0.022233
---- Total time: 0.3566
R0/1 is writing HOMO-4 cube
R0/1 is writing HOMO-3 cube
R0/1 is writing HOMO-2 cube
R0/1 is writing HOMO-1 cube
R0/1 is writing HOMO+0 cube
R0/1 is writing HOMO+1 cube
R0/1 is writing HOMO+2 cube
R0/1 is writing HOMO+3 cube
R0/1 is writing HOMO+4 cube
R0/1 is writing HOMO+5 cube
R0/1: finished, total time: 0.87s
*****************************

THE GENERATED FILES ARE:

S0_10_LUMO+4.cube  S0_3_HOMO-2.cube  S0_6_LUMO.cube    S0_9_LUMO+3.cube
S0_1_HOMO-4.cube   S0_4_HOMO-1.cube  S0_7

## Visualizing HOMO and LUMO separately


We note that the name of the HOMO file above, we copy it into "file" and we read the cube file and its energy (change the name of the file accordingly)

In [9]:
file = molecule + '_cubes/S0_5_HOMO.cube'
atoms = read(file)
a=!head -2 {file} | tail -1
b = str(a)
ene=(b[4:10])
view_homo=nv.NGLWidget()
caption_homo = "E="+ene+" eV\n"+"HOMO"
view_homo.add_component(nv.ASEStructure(atoms))
c_2 = view_homo.add_component(file)
c_2.clear()
c_2.add_surface(color='blue', isolevelType="value", isolevel=-0.01, opacity=0.05)
c_3 = view_homo.add_component(file)
c_3.clear()
c_3.add_surface(color='red', isolevelType="value", isolevel=0.01, opacity=0.05)


#
# And finally the visualization itself
#

print (caption_homo)
view_homo

E=-3.468 eV
HOMO


NGLWidget()

We do the same for the LUMO, we simply need to change the name of the file.

In [10]:
file = molecule + '_cubes/S0_6_LUMO.cube'
atoms = read(file)
a=!head -2 {file} | tail -1
b = str(a)
ene=(b[4:10])
view_lumo=nv.NGLWidget()
caption_lumo = "E="+ene+" eV\n"+'LUMO'
view_lumo.add_component(nv.ASEStructure(atoms))
c_2 = view_lumo.add_component(file)
c_2.clear()
c_2.add_surface(color='blue', isolevelType="value", isolevel=-0.01, opacity=0.05)
c_3 = view_lumo.add_component(file)
c_3.clear()
c_3.add_surface(color='red', isolevelType="value", isolevel=0.01, opacity=0.05)


#
# And finally the visualization itself
#
print (caption_lumo)
view_lumo

E=3.4684 eV
LUMO


NGLWidget()

We now create a combined view that visualizes orbital and energy:

In [11]:
import ipywidgets as widgets
widg_caption_homo = widgets.HTML(caption_homo)
combined_w_homo=widgets.HBox([view_homo,widg_caption_homo])

widg_caption_lumo = widgets.HTML(caption_lumo)
combined_w_lumo=widgets.HBox([view_lumo,widg_caption_lumo])


In [12]:
combined_w_homo

In [13]:
combined_w_lumo

## Visualizing all orbitals together

We will use a loop and arrays to caption all orbitals and plot a matrix of representations: see the file **show_orbitals.py** 

## Including the MATRIX VISUALIZATION OF ALL ORBITALS into a function

The function allows to choose the isosurface, and the first orbital to visualize in the array, as well as the last one. Note that for larger molecules (benzene) you better choose a few orbitals at the time, not to "kill" your jupyter.



In [14]:
import show_orbitals

In [19]:
#
# In this way, the molecule can be visualized with a call to the function followed by a call of the molecule itself
# See, in the Exercise directory, the file show_orbitals.py. Don't forget the total number of occupied orbitals (for naming)
#
!pwd
import importlib
importlib.reload(show_orbitals)
c2h2 = show_orbitals.all_orbitals(molecule,pk=my_pk,nhomo=1,nlumo=2,ntotocc=my_numberocc,nfirstview=0,nlastview=9,isosurf=0.01);


/home/jovyan/MMM_2024/Exercise_2025_04
Show Orbitals Version n.  4 ntotocc =  5
Archive:  ./cube-kit-pk58.zip
  inflating: cube-kit-pk58/BASIS_MOLOPT  
  inflating: cube-kit-pk58/aiida.inp  
  inflating: cube-kit-pk58/aiida.out  
  inflating: cube-kit-pk58/aiida.coords.xyz  
  inflating: cube-kit-pk58/aiida-RESTART.wfn  
R0/1, loading indexes (s0/1) 0:9 / 0:9
eval_cell_n:  [67 50 51]
loc_cell_n:  [71 70 70]
---- Setup: 0.0006
---- Radial calc time : 0.239485
---- Spherical calc time : 0.014436
---- Loc -> loc_morb time : 0.074127
---- loc_morb -> glob time : 0.027660
---- Total time: 0.3702
R0/1 is writing HOMO-4 cube
R0/1 is writing HOMO-3 cube
R0/1 is writing HOMO-2 cube
R0/1 is writing HOMO-1 cube
R0/1 is writing HOMO+0 cube
R0/1 is writing HOMO+1 cube
R0/1 is writing HOMO+2 cube
R0/1 is writing HOMO+3 cube
R0/1 is writing HOMO+4 cube
R0/1 is writing HOMO+5 cube
R0/1: finished, total time: 0.89s
S0_10_LUMO+4.cube
S0_1_HOMO-4.cube
S0_2_HOMO-3.cube
S0_3_HOMO-2.cube
S0_4_HOMO-1.cube
S0

In [21]:
# Now, magically, the name of the molecule will visuali


c2h2

## Now you are ready to generalize the exercise to the following molecules:

- H2
- CH4 (Methane)
- O2 (triplet state)
- Hexatriene
- Benzene 

### Note: for O2, you need a SPIN POLARIZED calculation (check the UKS option) with multiplicity 3 (also in the Orbital calculation)


You will get two sets of cube files, work only with the "S0" ones (spin up). Also, use n_homo 7 and n_lumo 5

### For Hexatriene vs. Benzene see the following [link](https://www.masterorganicchemistry.com/2017/05/05/the-pi-molecular-orbitals-of-benzene/)



# Assignments

1. For each molecule, draw a molecular orbital table filling the orbitals up to the correct level.
2. Discuss the difference between the H2 and O2 molecule.
3. Discuss the differences between the CH4 and CH2  and C6H6  molecules (hybridisation?)
4. Follow the discussion that you find in the link, and compare with your result. Discuss the differences you find between Hexatriene and Benzene
5. Which molecule has the largest Band Gap?
6. Apply a deformation to benzene and optimize again. Show the trajectory of the optimization. 